<a href="https://colab.research.google.com/github/dede0702/Homer-vs-Bart-CNN/blob/main/Homer_vs_Bart_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Introdução
Este projeto utiliza uma Rede Neural Convolucional (CNN) para classificar imagens dos personagens Homer e Bart, com base em um dataset de imagens previamente fornecido. O objetivo é construir um modelo que seja capaz de distinguir os dois personagens e, em seguida, testar a precisão do modelo, além de verificar a capacidade da rede neural de classificar corretamente novas imagens de Homer e Bart baixadas da internet.

Para isso, vamos abordar todas as etapas essenciais, como o pré-processamento das imagens, a criação da arquitetura da CNN e a avaliação da performance do modelo.

In [68]:
# Importando as bibliotecas necessárias
!pip install tensorflow
!pip install opencv-python
!pip install numpy
!pip install keras
!pip install matplotlib
!pip install tensorflow scikit-learn
import zipfile
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [69]:
# Caminho do arquivo zipado
zip_path = '/content/dataset_personagens.zip'

In [70]:
# Extraindo o arquivo zipado do dataset
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('dataset_personagens')

In [71]:
# Diretório do dataset contendo as pastas extraídas
dataset_dir = 'dataset_personagens/dataset_personagens'  # Ajuste o caminho conforme necessário
img_size = 64  # Definindo o tamanho das imagens

In [72]:
# Inicializando as listas para os dados e labels
X = []
y = []

# Carregamento das Imagens
 As imagens dos personagens Homer e Bart são carregadas de seus respectivos diretórios, redimensionadas e convertidas em arrays para serem processadas pela CNN.

In [73]:
for label, folder in enumerate(['training_set/homer', 'training_set/bart']):
    folder_path = os.path.join(dataset_dir, folder)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_size, img_size))  # Redimensionando para o tamanho definido
        X.append(img)
        y.append(label)  # Atribuindo 0 para Homer e 1 para Bart

In [74]:
# Convertendo as listas em arrays NumPy
X = np.array(X)
y = np.array(y)

# Normalização:
As imagens têm seus valores de pixel normalizados para que fiquem entre 0 e 1, o que melhora o desempenho do treinamento.

In [75]:
X = X / 255.0

In [76]:
# Convertendo as labels para categorias one-hot (Homer = [1, 0], Bart = [0, 1])
y = to_categorical(y, num_classes=2)

In [77]:
# Dividindo os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criação da CNN
A arquitetura da CNN foi projetada com três camadas de convolução,
cada uma seguida de uma camada de pooling para reduzir a dimensionalidade.


A camada Flatten achata a saída das camadas de convolução para que possam ser alimentadas na camada densa totalmente conectada, que faz a classificação.

In [78]:
model = Sequential()

In [79]:
# Primeira camada de convolução seguida de uma camada de pooling
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [80]:
# Segunda camada de convolução e pooling
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [81]:
# Terceira camada de convolução e pooling
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [82]:
# Flatten para achatar a saída e camada densa
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))  # Camada de saída para 2 classes (Homer, Bart)

In [83]:
# Compilando o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Treinamento e Avaliação:
O modelo é treinado por 10 épocas e validado com os dados de teste.
A acurácia final é impressa para análise de desempenho.

In [84]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.5359 - loss: 0.7638 - val_accuracy: 0.5750 - val_loss: 0.6754
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 61ms/step - accuracy: 0.6687 - loss: 0.6593 - val_accuracy: 0.5750 - val_loss: 0.6691
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.5732 - loss: 0.6606 - val_accuracy: 0.5750 - val_loss: 0.6458
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.6092 - loss: 0.5924 - val_accuracy: 0.6000 - val_loss: 0.6240
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.6678 - loss: 0.5338 - val_accuracy: 0.6500 - val_loss: 0.5944
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - accuracy: 0.8256 - loss: 0.4312 - val_accuracy: 0.8000 - val_loss: 0.5412
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.8226 - loss: 0.4086 - val_accuracy: 0.7250 - val_loss: 0.5105
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.7906 - loss: 0.4132 - val_accuracy: 0.8000 - val_loss: 0.4867
Ep

In [85]:
# Avaliando a acurácia do modelo
accuracy = model.evaluate(X_test, y_test)
print(f'Acurácia do modelo: {accuracy[1] * 100:.2f}%')

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7771 - loss: 0.4364
Acurácia do modelo: 77.50%


# Função de Previsão:
A função predict_image recebe o caminho de uma nova imagem e faz a previsão, retornando se é Homer ou Bart.

In [86]:
def predict_image(image_path):
    img = cv2.imread(image_path)

    # Verificando se a imagem foi carregada corretamente
    if img is None:
        print(f"Erro ao carregar a imagem: {image_path}")
        return None  # ou levantar uma exceção

    img = cv2.resize(img, (img_size, img_size))  # Redimensionando a imagem
    img = np.expand_dims(img, axis=0)  # Adicionando uma dimensão extra
    img = img / 255.0  # Normalizando a imagem
    prediction = model.predict(img)
    return 'Homer' if np.argmax(prediction) == 0 else 'Bart'

In [87]:
# Testando a previsão com uma nova imagem externa
image_path = '/content/SimpsonsandTrump.jpg'
prediction = predict_image(image_path)
print(f'O modelo previu: {prediction}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step
O modelo previu: Homer


#Conclusão
Neste projeto, foi desenvolvida uma Rede Neural Convolucional para classificar imagens de Homer e Bart. Após o treinamento, a CNN foi capaz de atingir uma acurácia significativa, sendo possível utilizar o modelo para classificar novas imagens dos personagens. Além disso, o código foi estruturado de forma a permitir ajustes em futuras experimentações, como o aumento de camadas, variação de parâmetros ou adição de novas imagens ao dataset.

Esse processo demonstrou como técnicas de deep learning podem ser aplicadas na classificação de imagens, proporcionando resultados precisos com o uso adequado de redes neurais convolucionais.